In [1]:
# import designMethods.en_13001_3_3 as en
# from output import create_output_file
from IPython.display import display, FileLink
import ipywidgets as widgets
import pathlib
import os
import itertools
from marsDemonstrator.gui.main_functions import Main_application

In [2]:
# !pip install voila
# !jupyter serverextension enable voila --sys-prefix

In [3]:
en_13001 = Main_application()

In [4]:
def init_gps(change):
    msg_box.value = "Initializing GPs: this may take up to 30 seconds"

    # initialize prediction class
    # en_13001.computed = en.Computed_data()

    # get current config (1 mast or 2 mast)
    en_13001.input.config = drop_config.value

    en_13001.init_gps()

    # en_13001["input"].config_loaded = True
    # en_13001.input.config_loaded = True
    msg_box.value = "Initialized GPs"

In [5]:
def start_computation(change):

    # if there were never inputs loaded throw error message 
    if en_13001.input_file_loaded is None:
        if uploader._counter == 0:
            msg_box.value = "Please upload an excel file"
            return
    
    # if there is a new upload file load its content and do make new computation
    if uploader._counter > 0:
        # initialize input class
        # en_13001.input = en.EN_input()

        # file to temporarily save input data to
        en_13001.input_file_path = pathlib.Path("./input.xlsx")
        with open(en_13001.input_file_path, "w+b") as i:
            i.write(uploader.data[-1])

        # read temp file and delete after
        fatal_error = en_13001.read_input_file(en_13001.input_file_path)

        if fatal_error:
            out_errors.clear_output()
            with out_errors:
                print("Fatal error in input file")
                print(fatal_error)
            uploader._counter = 0
            return

        os.remove(en_13001.input_file_path)

        # reset uploader after reading file
        uploader.value.clear()
        uploader._counter = 0
        
        # if gps for configuration were not loaded yet load them
        if en_13001.config_loaded is None:
            init_gps(None)
        msg_box.value = "Start Computation"

        en_13001.run_computation_and_create_output(1)

    msg_box.value = "Finished computation"

    # create output file
    local_file = FileLink(f"./{en_13001.outname.name}", result_html_prefix="Click here to download results: ")

    # display output file and write errors to output if there are any
    out_file.clear_output()
    out_errors.clear_output()
    with out_file:
        display(local_file)
        # display(btn_delete_output)
    write_error_reports(en_13001)

In [6]:
def write_error_reports(en_13001):
    with out_errors:
        for error in list(itertools.chain(*en_13001.input.error_report)):
            print(error)

In [7]:
# def delete_output_file(change):
#     os.remove(gen_vars["output_file"])
#     out_file.clear_output()

In [8]:
## Create widtges
# download link for input file
input_file = FileLink("./tests/input_file/inputparameters.xlsx", result_html_prefix="Click here to download example input file: ")

# uploader for input
uploader_label = widgets.Label("Upload input file:  ")
uploader = widgets.FileUpload(accept=".xlsx", multiple=False)
uploader_box = widgets.HBox([uploader_label, uploader])

# dropdown for computation mode
label_mode = widgets.Label("Computation Mode:  ")
drop_mode = widgets.Dropdown(options=["proof", "min_diameter"], value="proof")
mode_box = widgets.HBox([label_mode, drop_mode])

# dropdown for configuration
label_config = widgets.Label("SC Configuration: (m1: 1 Mast, m2: 2 Mast)  ")
drop_config = widgets.Dropdown(options=["m1", "m2"], value="m1")


# button for updating configuration
btn_update_config = widgets.Button(description="Update Configuration")
btn_update_config.on_click(init_gps)
config_box = widgets.HBox([label_config, drop_config, btn_update_config])

# button to start computation
btn_start = widgets.Button(description="Start computation")
btn_start.on_click(start_computation)

# output widget to display output file
out_file = widgets.Output()

# output widget to write error reports
out_errors = widgets.Output()

# message box for information on computation process
label_msg = widgets.Label("Message Box:   ")
msg_box = widgets.Text(value="")
msg_box_box = widgets.HBox([label_msg, msg_box])

# button to delete output file from server
# btn_delete_output = widgets.Button(description="Delete Resultfile")
# btn_delete_output.on_click(delete_output_file)

In [9]:
## Build gui
display(widgets.Label("MARS Demonstrator"))
display(input_file)
widgets.VBox([config_box, uploader_box, msg_box_box, btn_start, out_file, out_errors])

Label(value='MARS Demonstrator')

C:\Dokumente\MARS\mars-softwaredemonstrator\tests\input_file\inputparameters.xlsx